In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import sqlite3
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier


In [2]:
# connect to database

con = sqlite3.connect("employees_df")
cur = con.cursor()

In [3]:
employees = pd.read_csv("Resources/Employee.csv")
employees['EmployeeIndex'] = employees.index
employees

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,EmployeeIndex
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0,2
3,Masters,2016,Bangalore,3,27,Male,No,5,1,3
4,Masters,2017,Pune,3,24,Male,Yes,2,1,4
...,...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0,4648
4649,Masters,2013,Pune,2,37,Male,No,2,1,4649
4650,Masters,2018,New Delhi,3,27,Male,No,5,1,4650
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0,4651


In [4]:
predicated_df = employees[['EmployeeIndex', 'LeaveOrNot']]

cur.execute('CREATE TABLE IF NOT EXISTS Predicated (EmployeeIndex INTEGER, LeaveOrNot INTEGER)')
con.commit()

predicated_df.to_sql('Predicated', con, if_exists='replace', index = False)

cur.execute('''  
SELECT * FROM Predicated
          ''')

In [5]:
features_df = employees[['EmployeeIndex', 'Education', 'JoiningYear', 'City', 'PaymentTier', 'Age', 'Gender', 'EverBenched', 'ExperienceInCurrentDomain']]

cur.execute('CREATE TABLE IF NOT EXISTS Features (EmployeeIndex INTEGER, Education TEXT, JoiningYear TEXT, City TEXT, PaymentTier INTEGER, Age INTEGER, Gender TEXT, EverBenched TEXT, ExperienceInCurrentDomain INTEGER)')
con.commit()

features_df.to_sql('Features', con, if_exists='replace', index = False)

cur.execute('''  
SELECT * FROM Features
          ''')
con.commit()

In [ ]:
con = sqlite3.connect('employees_df')
cur = con.cursor()

In [6]:
# To turn database into dataframe

cur.execute('''CREATE TABLE EMPLOYEES as
SELECT * from FEATURES
JOIN Predicated
ON Features.EmployeeIndex = Predicated.EmployeeIndex; ''')
con.commit()

In [33]:
df = pd.read_sql_query("SELECT * FROM Employees", con)
employee_data_df = df.drop(['EmployeeIndex:1', 'EmployeeIndex'], axis=1)
employee_data_df

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1
...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4649,Masters,2013,Pune,2,37,Male,No,2,1
4650,Masters,2018,New Delhi,3,27,Male,No,5,1
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


In [35]:
employee_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Education                  4653 non-null   object
 1   JoiningYear                4653 non-null   int64 
 2   City                       4653 non-null   object
 3   PaymentTier                4653 non-null   int64 
 4   Age                        4653 non-null   int64 
 5   Gender                     4653 non-null   object
 6   EverBenched                4653 non-null   object
 7   ExperienceInCurrentDomain  4653 non-null   int64 
 8   LeaveOrNot                 4653 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 327.3+ KB


In [36]:
columns = ["Education", "JoiningYear", "City", "PaymentTier", "Age", "Gender", "EverBenched", "ExperienceInCurrentDomain"
]

target = ["LeaveOrNot"]

In [37]:
# Remove the `LeaveOrNot` loan status
LeaveOrNot_mask = employee_data_df['LeaveOrNot'] != 'LeaveOrNot'
employee_df = employee_data_df.loc[LeaveOrNot_mask]


# Split the Data into Training and Testing

In [38]:
# Create our features
# Create the training variables by converting the string values into numerical ones using the get_dummies() method.
X = pd.get_dummies(employee_data_df, columns=["Education", "City", "Gender", "EverBenched"]).drop("LeaveOrNot", axis=1)


# Create our target
y =employee_data_df["LeaveOrNot"]

In [39]:
X.describe()

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Bachelors,Education_Masters,Education_PHD,City_Bangalore,City_New Delhi,City_Pune,Gender_Female,Gender_Male,EverBenched_No,EverBenched_Yes
count,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000
mean,2015.062970,2.698259,29.393295,2.905652,0.773909,0.187621,0.038470,0.478831,0.248657,0.272512,0.402966,0.597034,0.897271,0.102729
std,1.863377,0.561435,4.826087,1.558240,0.418344,0.390451,0.192348,0.499605,0.432281,0.445300,0.490547,0.490547,0.303638,0.303638
min,2012.000000,1.000000,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2013.000000,3.000000,26.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,2015.000000,3.000000,28.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
75%,2017.000000,3.000000,32.000000,4.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,2018.000000,3.000000,41.000000,7.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [40]:
# Check the balance of our target values (1 = yes or 0 = no)
y.value_counts()

0    3053
1    1600
Name: LeaveOrNot, dtype: int64

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
Counter(y_train)

Counter({1: 1200, 0: 2289})

In [42]:
X_train.shape

(3489, 14)

# Balanced Random Forest Classifier

In [45]:
# Resample the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier

brfc = BalancedRandomForestClassifier(n_estimators =100, random_state=1)
brfc.fit(X_train, y_train)


BalancedRandomForestClassifier(random_state=1)

In [46]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7848625654450262

In [48]:
# Calculate confusion matrix.
cm = confusion_matrix(y_test, y_pred)
cm

array([[632, 132],
       [103, 297]], dtype=int64)

In [49]:
# Calculate accuracy score.
acc_score = balanced_accuracy_score(y_test, y_pred)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.7848625654450262


In [50]:
# imbalanced classification report

print( classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.86      0.83      0.74      0.84      0.78      0.62       764
          1       0.69      0.74      0.83      0.72      0.78      0.61       400

avg / total       0.80      0.80      0.77      0.80      0.78      0.62      1164



In [51]:
# List the features sorted in descending order by feature importance
importances = brfc.feature_importances_
sorted(zip(brfc.feature_importances_, X.columns), reverse=True)

[(0.3086914210245113, 'JoiningYear'),
 (0.19190147118381745, 'Age'),
 (0.09890156348096112, 'ExperienceInCurrentDomain'),
 (0.09325636092904764, 'PaymentTier'),
 (0.05727410779116132, 'City_Pune'),
 (0.054031392148976856, 'Education_Masters'),
 (0.046093613205681915, 'Gender_Male'),
 (0.03876883320695893, 'Education_Bachelors'),
 (0.03204738082388843, 'Gender_Female'),
 (0.025990119766903574, 'City_Bangalore'),
 (0.025687810889074857, 'City_New Delhi'),
 (0.009464044355872172, 'EverBenched_No'),
 (0.009311635638471467, 'EverBenched_Yes'),
 (0.008580245554672804, 'Education_PHD')]

In [52]:
print(f'Training Score: {brfc.score(X_train, y_train)}')
print(f'Testing Score: {brfc.score(X_test, y_test)}')

Training Score: 0.9120091716824305
Testing Score: 0.7981099656357389
